# anoGANs with noise


normal data : 0 ~ 4 숫자로 구성

anomalous data : 5 ~ 9 숫자로 구성

이미지 데이터는 64 by 64 크기는 -1 ~ 1

architecture는 DCGANs으로 구성


In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import time 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import pickle
import gzip


os.environ["CUDA_VISIBLE_DEVICES"]="1"


file_dir = 'anoGANs_MNIST_data/'

with gzip.open(file_dir + 'test_normal_data.pickle.gzip','rb') as f :
    test_normal_data = pickle.load(f)
    print('test_normal_data : ' ,test_normal_data.shape)

with gzip.open(file_dir + 'test_anomalous_data.pickle.gzip','rb') as f :
    test_anomalous_data = pickle.load(f)
    print('test_anomalous_data : ',test_anomalous_data.shape)
    
with gzip.open(file_dir + 'train_normal_data.pickle.gzip','rb') as f :
    train_normal_data = pickle.load(f)
    print('train_normal_data : ', train_normal_data.shape)
    
with gzip.open(file_dir + 'train_anomalous_data.pickle.gzip','rb') as f :
    train_anomalous_data = pickle.load(f)
    print('train_anomalous_data : ',train_anomalous_data.shape )



test_normal_data :  (5139, 64, 64, 1)
test_anomalous_data :  (4861, 64, 64, 1)
train_normal_data :  (28038, 64, 64, 1)
train_anomalous_data :  (26962, 64, 64, 1)


In [2]:
class DCGANs(object) :
    def __init__(self, sess, file_name,
                 train_lr = 0.001, train_beta1 = 0.5,train_epoch = 100, batch_size = 100, z_size = 100, 
                 ramda = 0.1, test_beta1 = 0.5, test_beta2=0.999,test_lr = 0.001, test_iteration = 1
                 keep_prob = 0.3) :
        
        self.sess = sess
        self.file_name = file_name
        self.train_lr = train_lr
        self.train_beta1 = train_beta1
        self.train_epoch = train_epoch
        self.batch_size = batch_size
        self.z_size = z_size
        self.ramda = ramda
        self.test_beta1 = test_beta1
        self.test_beta2 = test_beta2
        self.test_lr = test_lr
        self.test_iteration = test_iteration
        self.keep_prob =  keep_prob
        
        if not os.path.isdir(self.file_name) :
            os.mkdir(self.file_name)          
    def simple_G(self, x,x_cat, x_con, isTrain = True, reuse = False, name = 'G_out') : #input = (minibatch * w * h * ch)
    
        # out size = (in size + 2*padding - kenel)/strides + 1    
        w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
        b_init = tf.constant_initializer(0.0)
        
        with tf.variable_scope('G',reuse=reuse)  :

            #x = (-1, 1, 1, 100)
            x1 = tf.concat([x,x_cat, x_con],3)
            conv1 = tf.layers.conv2d_transpose(x1,1024,[4,4], strides=(1,1),padding = 'valid',
                    kernel_initializer=w_init, bias_initializer=b_init) 
            r1 = tf.nn.elu(tf.layers.batch_normalization(conv1,training=isTrain))#1024*4*4

            conv2 = tf.layers.conv2d_transpose(r1,512,[5,5], strides=(2,2),padding = 'same',
                    kernel_initializer=w_init, bias_initializer=b_init)
            r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#512*8*8

            conv3 = tf.layers.conv2d_transpose(r2,256,[5,5], strides=(2,2),padding = 'same',
                    kernel_initializer=w_init, bias_initializer=b_init)
            r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#256*16*16

            conv4 = tf.layers.conv2d_transpose(r3,128,[5,5], strides=(2,2),padding = 'same',
                    kernel_initializer=w_init, bias_initializer=b_init)
            r4 = tf.nn.elu(tf.layers.batch_normalization(conv4,training=isTrain))#128*32*32

            conv5 = tf.layers.conv2d_transpose(r4,1,[5,5], strides=(2,2),padding = 'same',
                    kernel_initializer=w_init, bias_initializer=b_init) #1*64*64

        r5= tf.nn.tanh(conv5,name=name)#1*64*64

        return r5
    def residual_loss_G(self, x, x_cat, x_con, isTrain = False, reuse = True, name = 'residual_loss_out') : #input = (minibatch * w * h * ch)
    
        # out size = (in size + 2*padding - kenel)/strides + 1    
        with tf.variable_scope('G',reuse=reuse) as scope :
            scope.reuse_variables()
            #x = (-1, 1, 1, 100)
            x1 = tf.concat([x,x_cat, x_con],3)
            conv1 = tf.layers.conv2d_transpose(x1,1024,[4,4], strides=(1,1),padding = 'valid')
            r1 = tf.nn.elu(tf.layers.batch_normalization(conv1,training=isTrain))#1024*4*4

            conv2 = tf.layers.conv2d_transpose(r1,512,[5,5], strides=(2,2),padding = 'same')
            r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#512*8*8

            conv3 = tf.layers.conv2d_transpose(r2,256,[5,5], strides=(2,2),padding = 'same')
            r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#256*16*16

            conv4 = tf.layers.conv2d_transpose(r3,128,[5,5], strides=(2,2),padding = 'same')
            r4 = tf.nn.elu(tf.layers.batch_normalization(conv4,training=isTrain))#128*32*32

            conv5 = tf.layers.conv2d_transpose(r4,1,[5,5], strides=(2,2),padding = 'same')

        r5= tf.nn.tanh(conv5,name=name)#1*64*64
        return r5
    def simple_D(self,x, self.keep_prob  isTrain=True,reuse = False) :

        w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
        b_init = tf.constant_initializer(0.0)
        with tf.variable_scope('D', reuse=reuse) :

            #x = (-1,64,64,1)
            # out size = (in size + 2*padding - kenel)/strides + 1   

            conv1 = tf.layers.conv2d(x,128,[5,5], strides=(2,2),padding = 'same',
                                    kernel_initializer=w_init, bias_initializer=b_init) 
            r1 = tf.nn.elu(conv1)#128*32*32

            conv2 = tf.layers.conv2d(r1,256,[5,5], strides=(2,2),padding = 'same',
                                    kernel_initializer=w_init, bias_initializer=b_init)
            r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#256*16*16

            conv3 = tf.layers.conv2d(r2,512,[5,5], strides=(2,2),padding = 'same',
                                    kernel_initializer=w_init, bias_initializer=b_init)
            r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#512*8*8

            
            r3_flatten = tf.reshape(r3, [-1,512*8*8])  # 512*8*8 = 32768
            
            conv4 = tf.layers.conv2d(r3,1024,[5,5], strides=(2,2),padding = 'same',
                                    kernel_initializer=w_init, bias_initializer=b_init)
            r4 = tf.nn.elu(tf.layers.batch_normalization(conv4,training=isTrain))#1024*4*4
            
            
            conv5 = tf.layers.conv2d(r4,1,[4,4], strides=(1,1),padding = 'valid',
                                    kernel_initializer=w_init, bias_initializer=b_init) 
        
        with tf.variable_scope('Q', reuse=reuse) :
            # cat
            w1_cat = tf.get_variable('w1_cat',[r3_flatten.get_shape()[1], 512initializer=w_init)
            b1_cat = tf.get_variable('b1_cat',[512],initializer=b_init)
            h1_cat = tf.nn.dropout(tf.nn.elu(tf.matmul(r3_flatten,w1_cat) + b1_cat), keep_prob)
            
            w2_cat = tf.get_variable('w2_cat',[h1_cat.get_shape()[1], 5],initializer=w_init)
            b2_cat = tf.get_variable('b2_cat',[5],initializer=b_init)
           
            # con
            w1_con = tf.get_variable('w1_con',[r3_flatten.get_shape()[1], 512initializer=w_init)
            b1_con = tf.get_variable('b1_con',[512],initializer=b_init)
            h1_con = tf.nn.dropout(tf.nn.elu(tf.matmul(r3_flatten,w1_con) + b1_con), keep_prob)
            
            w2_con = tf.get_variable('w2_con',[h1_cat.get_shape()[1], 5],initializer=w_init)
            b2_con = tf.get_variable('b2_con',[5],initializer=b_init)

        r5 = tf.nn.sigmoid(conv5)#1*1*1
        h2_cat = tf.nn.softmax(tf.matmul(h1_cat,w1_cat) + b1_cat)
        h2_con = tf.nn.sigmoid(tf.matmul(h1_con,w1_con) + b1_con)
        return r5, h2_cat, h2_con
                                               
    def feature_extractor_D(self, x, isTrain=False, reuse = True, name='feature_out') :

        with tf.variable_scope('D', reuse=reuse) as scope :
            scope.reuse_variables()
            #x = (-1,28,28,1)

            conv1 = tf.layers.conv2d(x,128,[5,5], strides=(2,2),padding = 'same') 
            r1 = tf.nn.elu(conv1)#128*32*32

            conv2 = tf.layers.conv2d(r1,256,[5,5], strides=(2,2),padding = 'same')
            r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#256*16*16

            conv3 = tf.layers.conv2d(r2,512,[5,5], strides=(2,2),padding = 'same')
        r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain),name=name)#512*8*8
        
        return r3
    def train(self, data_set) : 
        

        
        z = tf.placeholder(tf.float32,shape=(None,1,1,self.z_size),name = 'z')
        z_cat = tf.placeholder(tf.float32,shape=(None,1,1,5),name = 'z_cat')                                           
        z_con = tf.placeholder(tf.float32,shape=(None,1,1,5),name = 'z_con')                                        
                                               
        u = tf.placeholder(tf.float32, shape = (None, 64,64,1),name='u')

        keep_prob  = tf.placeholder(tf.float32,name='keep_prob')
                                               
                                               
        test_u = tf.placeholder(tf.float32, shape = (None, 64,64,1),name='test_u')
        latent_z = tf.placeholder(tf.float32,shape=(None,1,1,self.z_size),name = 'latent_z')    

        isTrain = tf.placeholder(dtype=tf.bool,name='isTrain') 

        soft_one = tf.placeholder(tf.float32, shape = (None, 1,1,1),name='soft_one') # 0.9~1.0
        soft_zero = tf.placeholder(tf.float32, shape = (None, 1,1,1),name='soft_zero') # 0.0~0.1

        ramda = tf.placeholder(tf.float32,name='ramda') # 0.0~0.1

        G_z = self.simple_G(z,name='G_z')

        D_real, _, _ = self.simple_D(u,  keep_prob , isTrain)
        D_fake, cat_fake, con_fake = self.simple_D(G_z , keep_prob ,isTrain, reuse=True)

        query_z =  self.residual_loss_G(latent_z, reuse = True, name ='query_z')

        discrimination_from_query_z= self.simple_D(query_z, isTrain=False, reuse=True)

        feature_u = self.feature_extractor_D(test_u, reuse = True, name ='feature_y')
        feature_z = self.feature_extractor_D(query_z, reuse = True, name ='feature_z')

        D_real_loss = tf.reduce_mean(-(soft_one*tf.log(D_real + 1e-8) + (1-soft_one)*tf.log(1- D_real + 1e-8)), name = 'D_real_loss')
        D_fake_loss = tf.reduce_mean(-(soft_zero*tf.log(D_fake + 1e-8) + (1-soft_zero)*tf.log(1 - D_fake + 1e-8)), name = 'D_fake_loss')
        cat_fake_loss = tf.reduce_mean(-z_cat*tf.log(cat_fake + 1e-8),name = 'cat_fake_loss')
        con_fake_loss = tf.reduce_mean(-(z_con*tf.log(con_fake+1e-8) + (1-z_con)*tf.log(1- con_faje + 1e-8)),name = 'con_fake_loss')
                                               
                                               
        D_loss =  tf.add(D_real_loss, D_fake_loss,name='D_loss')
        G_loss =  tf.reduce_mean(-tf.log(D_fake + 1e-8),name='G_loss')

        residual_loss = tf.reduce_mean(tf.abs(query_z - test_u),axis = (1,2,3), name = 'residual_loss')
        feature_loss = tf.reduce_mean(tf.abs(feature_u-feature_z),axis = (1,2,3),name = 'feature_loss')
        discrimination_loss = tf.reduce_mean(-tf.log(discrimination_from_query_z+ 1e-8),axis = (1),name = 'discrimination_loss')

        mapping_loss = tf.add((1-ramda)*residual_loss, ramda*feature_loss,name = 'mapping_loss') 

        T_vars = tf.trainable_variables()
        D_vars = [var for var in T_vars if var.name.startswith('D')]
        G_vars = [var for var in T_vars if var.name.startswith('G')]
        Q_vars = [var for var in T_vars if var.name.startswith('Q')]
            # When using the batchnormalization layers,
            # it is necessary to manually add the update operations
            # because the moving averages are not included in the graph

        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)) :        
            D_optim = tf.train.AdamOptimizer(self.train_lr, beta1=self.train_beta1).minimize(D_loss, var_list=D_vars, name='D_optim') 
            G_optim = tf.train.AdamOptimizer(self.train_lr, beta1=self.train_beta1).minimize(G_loss + cat_fake_loss + con_fake_loss,
                                                                                             var_list=G_vars, name='G_optim')
            Q_optim = tf.train.AdamOptimizer(self.train_lr, beta1=self.train_beta1).minimize(G_loss + cat_fake_loss + con_fake_loss,
                                                                                             var_list=Q_vars, name='Q_optim')

        grad_z = tf.gradients(mapping_loss, latent_z, name ='grad_z')
        #print(grad_z)
        
        self.sess.run(tf.global_variables_initializer())

        np.random.seed(int(time.time()))

        test_z = np.random.normal(0,1,size=(16,1,1,self.z_size))
        
        log_txt = open(self.file_name +'/log.txt','w')

        hist_G = []
        hist_D = []
        G_error = []
        D_error = []
        D_fake_error = []
        D_real_error = []

        start = time.time()
        for epoch in range(self.train_epoch) :

            np.random.seed(int(time.time()))
            train_data = idx_shuffle(data_set) 

            for iteration in range(train_data.shape[0] // self.batch_size) : 

                train_images = train_data[iteration*self.batch_size : (iteration+1)*self.batch_size]      
                u_ = np.reshape(train_images,(-1,64,64,1)) 
                z_ = np.random.normal(0,1,size=(self.batch_size,1,1,self.z_size))
                z_cat_ = np.random.normal(0,1,size=(self.batch_size,1,1,5))                                  
                z_con_ = np.random.normal(0,1,size=(self.batch_size,1,1,5))
                
                soft_one_ = np.random.uniform(0.9,1.0,(self.batch_size,1,1,1))
                soft_zero_ = np.random.uniform(0.0,0.1,(self.batch_size,1,1,1))

                _ , D_e,D_real_e,D_fake_e = self.sess.run([D_optim, D_loss,D_real_loss,D_fake_loss], {u : u_, z : z_, isTrain : True,
                                                                                              z_cat : z_cat_, z_con : z_con_, soft_one : soft_one_, soft_zero : soft_zero_})
                D_error.append(D_e)
                D_real_error.append(D_real_e)
                D_fake_error.append(D_fake_e)

                #    train_images,train_labels = mnist.train.next_batch(100)    
                #    u_ = np.reshape(train_images,(-1,64,64,1)) 
                #    z_ = np.random.normal(0,1,size=(100,1,1,100))

                _ , G_e = self.sess.run([G_optim, G_loss], {u : u_, z : z_, isTrain : True, z_cat : z_cat_, z_con : z_con_}) 
                G_error.append(G_e)

            hist_D.append(np.mean(D_error)) 
            hist_G.append(np.mean(G_error))

            print('D_e : %.6f, D_real_e : %.6f, D_fake_e : %.6f, G_e : %.6f'%(np.mean(D_error), np.mean(D_real_error),
                np.mean(D_fake_error), np.mean(G_error)))
            log_txt.write('D_e : %.6f, D_real_e : %.6f, D_fake_e : %.6f, G_e : %.6f\n'%(np.mean(D_error),
                np.mean(D_real_error), np.mean(D_fake_error), np.mean(G_error)))

            r = self.sess.run([G_z],feed_dict={z : test_z, isTrain : False})        
            mnist_4by4_save(np.reshape(r,(-1,64,64,1)),self.file_name + '/result_{}.png'.format(str(epoch).zfill(3)))

            G_errer = []
            D_errer = []
            D_fake_error = []
            D_real_error = []          
    def anomaly_detect(self, data_set) :

        new_saver = tf.train.import_meta_graph(file_name + '/para.cktp.meta')
        new_saver.restore(sess, tf.train.latest_checkpoint(file_name + '/'))


        z = sess.graph.get_tensor_by_name("z:0")
        u = sess.graph.get_tensor_by_name("u:0")

        noise = sess.graph.get_tensor_by_name("noise:0")

        test_u = sess.graph.get_tensor_by_name("test_u:0")
        latent_z = sess.graph.get_tensor_by_name("latent_z:0")

        isTrain = sess.graph.get_tensor_by_name("isTrain:0")

        soft_one =sess.graph.get_tensor_by_name("soft_one:0") # 0.9~1.0
        soft_zero = sess.graph.get_tensor_by_name("soft_zero:0") # 0.0~0.1

        ramda = sess.graph.get_tensor_by_name("ramda:0")

        G_z = sess.graph.get_tensor_by_name("G_z:0")
        query_z = sess.graph.get_tensor_by_name("query_z:0")


        feature_u = sess.graph.get_tensor_by_name('feature_y:0')
        feature_z = sess.graph.get_tensor_by_name('feature_z:0')

        D_real_loss = sess.graph.get_tensor_by_name('D_real_loss:0')
        D_fake_loss = sess.graph.get_tensor_by_name('D_fake_loss:0')

        D_loss = sess.graph.get_tensor_by_name("D_loss:0")
        G_loss = sess.graph.get_tensor_by_name("G_loss:0")

        residual_loss = sess.graph.get_tensor_by_name("residual_loss:0")
        feature_loss = sess.graph.get_tensor_by_name("feature_loss:0")
        discrimination_loss = sess.graph.get_tensor_by_name("discrimination_loss:0")
        mapping_loss = sess.graph.get_tensor_by_name("mapping_loss:0")

        D_optim = sess.graph.get_operation_by_name("D_optim")
        G_optim = sess.graph.get_operation_by_name("G_optim")

        grad_z =  sess.graph.get_tensor_by_name("grad_z/G_1/conv2d_transpose/conv2d_transpose_grad/Conv2D:0")

        ano_score = []
        fea_score = []
        res_score = []
        for iteration in range(data_set.shape[0] // self.batch_size) : 

            test_images = data_set[iteration*self.batch_size : (iteration+1)*self.batch_size]      

            np.random.seed(int(time.time()))
            latent_z_ = np.random.normal(0,1,size=(self.batch_size,1,1,self.z_size))

            adam_m = np.zeros((self.batch_size,1,1,self.z_size))
            adam_v = np.zeros((self.batch_size,1,1,self.z_size))
            beta1_t = self.test_beta1
            beta2_t = self.test_beta2
      
            for i in range(self.test_iteration) :  
                grad  = sess.run(grad_z, {test_u : np.reshape(test_images,(-1,64, 64,1)), latent_z : latent_z_, ramda : self.ramda })

                lr_t = self.test_lr*np.sqrt(1-beta2_t)/(1-beta1_t)
                adam_m = self.test_beta1*adam_m + (1-self.test_beta1)*grad
                adam_v = self.test_beta2*adam_v + (1-self.test_beta2)*grad*grad
                latent_z_ = latent_z_ - lr_t*adam_m/(np.sqrt(adam_v)+1e-8)
                beta1_t = beta1_t*self.test_beta1
                beta2_t = beta2_t*self.test_beta2

            map_loss, fea_loss, res_loss = sess.run([mapping_loss, feature_loss,residual_loss],
                                                             {test_u : np.reshape(test_images,(-1,64, 64,1)), latent_z : latent_z_, ramda : self.ramda })
            ano_score = np.append(ano_score, map_loss)
            fea_score = np.append(fea_score, fea_loss)
            res_score = np.append(res_score, res_loss)

            print(iteration ,data_set.shape[0] // self.batch_size)

        np.savetxt(self.file_name + '/ano_score.txt', ano_score, fmt='%6f') 
        np.savetxt(self.file_name + '/fea_score.txt', fea_score, fmt='%6f') 
        np.savetxt(self.file_name + '/res_score.txt', res_score, fmt='%6f')         
        
def idx_shuffle(x) : 
    l = x.shape[0]
    idx = np.arange(l)
    np.random.shuffle(idx)
    shuffled_x = np.empty(x.shape)

    for i in range(l):
        shuffled_x[idx[i]] = x[i]
    
    return shuffled_x
def mnist_4by4_save(samples,path):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)    
    gs.update(wspace=0.05, hspace=0.05) #이미지 사이간격 조절
  
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')    
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
   
        plt.imshow(sample.reshape(64, 64), cmap='Greys_r',clim=(0.0,1.0))
    plt.colorbar()
    plt.savefig(path, bbox_inches='tight')
    plt.close(fig)
   
    return None
def gan_loss_graph_save(G_loss,D_loss,path):
    x1 = range(len(G_loss))
    x2 = range(len(D_loss))
      
    y1 = G_loss
    y2 = D_loss
  
      
    plt.plot(x1,y1,label='G_loss') 
    plt.plot(x2,y2,label='D_loss') 
  
    plt.xlabel('weight per update')
    plt.ylabel('loss')             
    plt.legend(loc=4)              
    plt.grid(True)
    plt.tight_layout()
  
    plt.savefig(path)              

    return None

    
    

In [3]:
file_name = 'ex_ano_infoGANs_1'

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess :
    
    dcgan = DCGANs(sess,file_name)
    dcgan.train(train_normal_data)
    #dcgan.anomaly_detect(test_normal_data)

D_e : 3.706472, D_real_e : 1.317826, D_fake_e : 2.388646, G_e : 3.971218
D_e : 2.701553, D_real_e : 0.850282, D_fake_e : 0.846353, G_e : 2.665395
D_e : 2.341212, D_real_e : 0.813723, D_fake_e : 0.806806, G_e : 2.216629
D_e : 2.151100, D_real_e : 0.794141, D_fake_e : 0.786622, G_e : 1.984245
D_e : 2.027459, D_real_e : 0.772468, D_fake_e : 0.760428, G_e : 1.852805
D_e : 1.916654, D_real_e : 0.710450, D_fake_e : 0.652177, G_e : 1.848310
D_e : 1.813689, D_real_e : 0.622746, D_fake_e : 0.573159, G_e : 1.857493
D_e : 1.752507, D_real_e : 0.686639, D_fake_e : 0.637589, G_e : 1.845353
D_e : 1.658555, D_real_e : 0.464055, D_fake_e : 0.442891, G_e : 1.812863
D_e : 1.558899, D_real_e : 0.347056, D_fake_e : 0.314931, G_e : 1.712734
D_e : 1.468148, D_real_e : 0.286165, D_fake_e : 0.274481, G_e : 1.591973
D_e : 1.398773, D_real_e : 0.335207, D_fake_e : 0.300430, G_e : 1.506819
D_e : 1.323792, D_real_e : 0.211079, D_fake_e : 0.212950, G_e : 1.393874
D_e : 1.277693, D_real_e : 0.350071, D_fake_e : 0.3

In [1]:
512*8*8

32768